In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import collections as cll

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import pandas.plotting as pd_plot
from pandas.plotting import scatter_matrix


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.



In [ ]:
files = os.listdir("../input")
print( files[0])
d = pd.read_csv('../input/' + 'train_V2.csv')


# facciamo 2 colonne : 
* NUM OF TEAM MATES 
* NUM OF TOTAL PEOPLE 4EC MATCH

In [ ]:
# #  NUM OF TOTAL PEOPLE 4EC MATCH
# # a = d['matchId']
# # tot_people_4ec_match = d['matchId'].value_counts().sort_index(0)

# # hashmap 
# c= cll.Counter(d.matchId)
# # map by hash
# tot_people_4ec_match = d.matchId.map(c)
# # new column containing for each entry the number of players for each match
# # print(tot_people_4ec_match)

# PG
d['tot_people_4ec_match'] = d.groupby('matchId')['matchId'].transform('count')
# d['mean_num_of_mates_per_group'] = d['tot_people_4ec_match'] / d['numGroups']

In [ ]:
# # NUM OF TEAM MATES
# # num_of_team_mates = d['groupId'].value_counts().sort_index(0)
# # d['groupId'].apply(lambda x: x.value_counts())
# # print(d.head())

# # hashmap 
# c1= cll.Counter(d.groupId)
# # map by hash
# n_of_team_m8 = d.groupId.map(c1)
# # new column containing for each entry the number of friendly players (same team)
# #print(n_of_team_m8)

# PG
d['num_of_team_mates'] = d.groupby('groupId')['groupId'].transform('count')
# d['gain_team_mates'] = d['num_of_team_mates']/d['mean_num_of_mates_per_group']

In [ ]:
# d.loc['matchId']

CORRELAZIONE TRA I DATI SOLO NUMERICI

In [ ]:
def plot_corr_m(d_arg, bPlot):
    d_numeric = d_arg.select_dtypes(include=['number'])
    #print(d_numeric.head)
    corr_mx = d_numeric.corr()

    # Questo un modo per undock-are la figure, ma funziona per matplotlib, dicono.... 
    #%matplotlib qt
    if bPlot:
        aa = scatter_matrix(corr_mx[d_numeric.columns],figsize=(40,30))
    
    return corr_mx
    
# plot_corr_m(d[['num_of_team_mates','tot_people_4ec_match','gain_team_mates','winPlacePerc']],True)



In [ ]:
# d = d.drop(columns= ['mean_num_of_mates_per_group','tot_people_4ec_match'])

In [ ]:
# #  normalizzazione
def Normalizer(m_numeric):
    min_max_scaler = preprocessing.MinMaxScaler()

    only_val = m_numeric.values.astype(np.float64)
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(only_val)

    # Run the normalizer on the dataframe
    m_normalized = pd.DataFrame(x_scaled)
    return m_normalized

#     d_num_norm = d_normalized.select_dtypes(include=['number'])
    #print(d_numeric.head)
#     corr_mx = d_num_norm.corr()

In [ ]:
# # ADD the new two columns
# d['people4EachTeam'] = tot_people_4ec_match
# d['numberTeamMate'] = n_of_team_m8

In [ ]:
def matchTypeToNum(d):
    encoder = preprocessing.LabelEncoder()
    matchType = d["matchType"]
#     matchTypeEn = encoder.fit_transform(matchType)
    d.matchType = matchType
    return d

d = matchTypeToNum(d)

In [ ]:
# d2 = d
# d2['newfeat'] = d2.weaponsAcquired/d2.walkDistance
# plot_corr_m(d2[['newfeat','weaponsAcquired','walkDistance','winPlacePerc']],True)

In [ ]:
# Drop columns with correlation or deperecated
# d1 = d.drop(columns= ['killStreaks', 'maxPlace', 'longestKill'])
d1= d

In [ ]:
to_bin_cols = ['matchDuration','damageDealt','DBNOs','heals','revives','weaponsAcquired','vehicleDestroys']

def quantize_all(dataset,to_bin_cols):
    
    label = LabelEncoder()
    for col in to_bin_cols:    
        dataset[col] = pd.qcut(dataset[col],3,duplicates = 'drop')    
        dataset[col] = label.fit_transform(dataset[col])
    return dataset

# d1 = quantize_all(d1,to_bin_cols)

In [ ]:
#at the moment I'll drop also winPoints and rankPoints
# but i'm taking them in consideration for further feature construction
# d2 = d1.drop(columns = ['rankPoints'])
#d2_cm = plot_corr_m(d2)

In [ ]:
# I'll drop also kill for it is less specific than precious kill-type feature.
# and also weaponAcquired  for it is corr 
#     maxDistance-weaponsAcquired 
#     heals- weaponAcquired
# d3 = d2.drop(columns = ['kills','weaponsAcquired'])
#d3_cm = plot_corr_m(d3)
d3= d2

In [ ]:
d.drop(columns=['matchId','groupId','Id','matchType'],inplace = True)
# d3 =d3[d3.walkDistance >10]

In [ ]:
# d3.loc[d3.matchDuration<180].head(10)
# print(d3.iloc[4446682])
d3.loc[d3.matchId == '0ac026a48eb471'].head(10)
# guarda i primi tre in killPlace che compagni hanno

In [ ]:
# print(d3.iloc[4446682])
# d3.loc[d3.groupId == '4e5b8eefdd669e'].head()
# guarda i primi tre in killPlace che  ha

In [ ]:
D = d.dropna(subset=['winPlacePerc'])
D_samples = D.drop(columns=['winPlacePerc'])
D.set_axis(D.columns,axis = 'columns',inplace=True)
D_label = d.winPlacePerc.dropna()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(D_samples, D_label)

In [ ]:
test = pd.read_csv('../input/' + files[1])
# c= cll.Counter(test.matchId)
# map by hash
# tot_people_4ec_match = test.matchId.map(c)
# c1= cll.Counter(test.groupId)
# map by hash
# n_of_team_m8 = test.groupId.map(c1)

# ADD the new two columns
test['tot_people_4ec_match'] = d.groupby('matchId')['matchId'].transform('count')
test['num_of_team_mates'] = test.groupby('groupId')['groupId'].transform('count') # PANDAS way to do make


test['mean_num_of_mates_per_group'] = test['tot_people_4ec_match'] / test['numGroups']
test['gain_team_mates'] = test['num_of_team_mates']/test['mean_num_of_mates_per_group']
test = matchTypeToNum(test)

test = quantize_all(test)

# Drop columns with correlation or deperecated
t3 = test.dropna()
result_Id = t3.Id
t3 = t3.drop(columns= ['killStreaks', 'maxPlace', 'longestKill','winPoints', 'rankPoints','kills','weaponsAcquired','matchId','groupId','Id'])
T = Normalizer(t3)
T.set_axis(t3.columns,axis = 'columns',inplace=True)



In [ ]:
results = forest_reg.predict(T)
# dfresult = pd.DataFrame(results,columns = ['maxPlacePerc'])
dfresult = pd.DataFrame( result_Id)
dfresult = dfresult.assign(winPlacePerc=results)
dfresult['winPlacePerc']= results

In [ ]:
fid = open('result.csv','w')
dfresult.to_csv(fid,index = False)